# 3. Divergence

Next we will translate a divergence stencil. We approximate the divergence of a vector field $\mathbf{v}$ at the middle point of a cell $\mathbf{P}$ in the following way: We take the dot product of the normal velocity $\mathbf{n}_e$ of each direct neighbor edge of $\mathbf{P}$  with $\mathbf{v}_e$ which is multipled with the edge length $L_e$. The contributions from all three edges of a cell are summed up and then divided by the area of the cell $A_P$. In the next pictures we can see a graphical representation of all of the quantities involved:

![](../images/divergence_picture.png "Divergence")

And the equation:

![](../images/divergence_formula.png "Divergence")

The orientation of the edge has to factor in, since we do not know, in general, if the normal of an edge is pointed inwards or outwards of any cell we are looking at. We cannot have only outwards pointing edge normals, because if we look at two neighboring cells, the normal of their shared edge has to point outwards for one of the cells, but inwards for the other.

![](../images/edge_orientation.png "Edge Orientation")


In [6]:
from helpers import *

In [7]:
def divergence_numpy(
    c2e: np.array,
    u: np.array,
    v: np.array,
    nx: np.array,
    ny: np.array,
    L: np.array,
    A: np.array,
    edge_orientation: np.array,
) -> np.array:
    uv_div = np.sum((u[c2e] * nx[c2e] + v[c2e] * ny[c2e]) * L[c2e] * edge_orientation, axis=1) / A
    return uv_div

In [8]:
@gtx.field_operator
def divergence(
    u: gtx.Field[Dims[E], float],
    v: gtx.Field[Dims[E], float],
    nx: gtx.Field[Dims[E], float],
    ny: gtx.Field[Dims[E], float],
    L: gtx.Field[Dims[E], float],
    A: gtx.Field[Dims[C], float],
    edge_orientation: gtx.Field[Dims[C, C2EDim], float],
) -> gtx.Field[Dims[C], float]:
    uv_div = A

    return uv_div

In [ ]:
def test_divergence():
    backend = None
    # backend = gtfn_cpu
    # backend = gtfn_gpu

    cell_domain = gtx.domain({C: n_cells})
    edge_domain = gtx.domain({E: n_edges})

    u = random_field(edge_domain, allocator=backend)
    v = random_field(edge_domain, allocator=backend)
    nx = random_field(edge_domain, allocator=backend)
    ny = random_field(edge_domain, allocator=backend)
    L = random_field(edge_domain, allocator=backend)
    A = random_field(cell_domain, allocator=backend)
    edge_orientation = random_sign(gtx.domain({C: n_cells, C2EDim: 3}), allocator=backend)

    divergence_ref = divergence_numpy(
        c2e_table,
        u.asnumpy(),
        v.asnumpy(),
        nx.asnumpy(),
        ny.asnumpy(),
        L.asnumpy(),
        A.asnumpy(),
        edge_orientation.asnumpy(),
    )

    c2e_connectivity = gtx.as_connectivity([C, C2EDim], codomain=E, data=c2e_table)

    divergence_gt4py = gtx.zeros(cell_domain, allocator=backend)

    divergence(
        u,
        v,
        nx,
        ny,
        L,
        A,
        edge_orientation,
        out=divergence_gt4py,
        offset_provider={C2E.value: c2e_connectivity},
    )

    assert np.allclose(divergence_gt4py.asnumpy(), divergence_ref)

In [10]:
test_divergence()
print("Test successful")

AssertionError: 

## 3. Divergence in ICON

In ICON we can find a divergence in diffusion which looks somewhat like this, but also quite a bit different:

```fortran
      DO jb = i_startblk,i_endblk

        CALL get_indices_c(p_patch, jb, i_startblk, i_endblk, &
                           i_startidx, i_endidx, rl_start, rl_end)
        DO jk = 1, nlev
          DO jc = i_startidx, i_endidx

            div(jc,jk) = p_nh_prog%vn(ieidx(jc,jb,1),jk,ieblk(jc,jb,1))*p_int%geofac_div(jc,1,jb) + &
                         p_nh_prog%vn(ieidx(jc,jb,2),jk,ieblk(jc,jb,2))*p_int%geofac_div(jc,2,jb) + &
                         p_nh_prog%vn(ieidx(jc,jb,3),jk,ieblk(jc,jb,3))*p_int%geofac_div(jc,3,jb)
          ENDDO
        ENDDO
      ENDDO
```

Two assumptions are necessary to derive the ICON version of the divergence starting from our version above:
* Assume that the velocity components $u$ is always orthogonal and the velocity component $v$ is always parallel to the edge, in ICON these are called $vn$ and $vt$ where the n stands for normal and the t for tangential.
* At ICON startup time merge all constants (such as cell area $A_P$ and edge length $L_e$) into one array of geometrical factors `p_int%geofac_div`, which are constant during time stepping:

```fortran
    DO jb = i_startblk, i_endblk

      CALL get_indices_c(ptr_patch, jb, i_startblk, i_endblk, &
        & i_startidx, i_endidx, rl_start, rl_end)

      DO je = 1, ptr_patch%geometry_info%cell_type
        DO jc = i_startidx, i_endidx

          ile = ptr_patch%cells%edge_idx(jc,jb,je)
          ibe = ptr_patch%cells%edge_blk(jc,jb,je)

          ptr_int%geofac_div(jc,je,jb) = &
            & ptr_patch%edges%primal_edge_length(ile,ibe) * &
            & ptr_patch%cells%edge_orientation(jc,jb,je)  / &
            & ptr_patch%cells%area(jc,jb)

        ENDDO !cell loop
      ENDDO

    END DO !block loop

```